## データのロード

In [1]:
import pandas as pd

df = pd.read_csv("yomilog.csv")
df

,name,title,price
0,yukie,たのしいPython,3000
1,altnight,たのしいPython,3000
2,furi,たのしいPython,3000
3,mtb,たのしいPython,3000
4,susumuis,おいしいPython,1000
5,altnight,やさしいNumPy,4000
6,yukie,やさしいNumPy,4000
7,mtb,やさしいNumPy,4000
8,yukie,難しい機械学習,2000
9,mtb,難しい機械学習,2000


## 最適化の確認

まずは最適化が実行できるか検証。

### susumuis がまだ買っていない書籍のリスト

In [2]:
# 既読の書籍
titles_set = df.loc[df["name"] == "susumuis", "title"].unique()
# 未読の書籍
df_filtered = df[~df["title"].isin(titles_set)]

### 書籍ごとのタイトル、価格、読んだ人数を集計

In [3]:
df_books = (
    df_filtered.groupby("title")
    .agg({"name": "nunique", "price": "first"})
    .reset_index()
    .rename(columns={"name": "n_readers"})
)
df_books

,title,n_readers,price
0,たのしいPython,4,3000
1,やさしいNumPy,3,4000
2,難しい機械学習,2,2000


### モデルと変数を定義

In [4]:
from mip import Model

m = Model()
# 変数: 書籍をおすすめする=1, しない=0
df_books["Var_x"] = m.add_var_tensor((len(df_books),), "x", var_type="B")
df_books

,title,n_readers,price,Var_x
0,たのしいPython,4,3000,x_0
1,やさしいNumPy,3,4000,x_1
2,難しい機械学習,2,2000,x_2


### 目的関数と制約条件

In [5]:
from mip import maximize, xsum

# 目的関数: 同僚の多くが読んでいる書籍をなるべく多くおすすめする
m.objective = maximize(xsum(df_books["n_readers"] * df_books["Var_x"]))
# 制約条件: 金額が予算以内
m += xsum(df_books["price"] * df_books["Var_x"]) <= 5000

### 最適化実行

In [6]:
m.optimize()

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 1 (0) rows, 3 (0) columns and 3 (0) elements
Clp1000I sum of infeasibilities 0 - average 0, 3 fixed columns
Coin0506I Presolve 0 (-1) rows, 0 (-3) columns and 0 (-3) elements
Clp0000I Optimal - objective value -0
Clp0000I Optimal - objective value -0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0000I Optimal - objective value 6
Clp0000I Optimal - objective value 6
Clp0000I Optimal - objective value 6
Clp0032I Optimal objective 6 - 0 iterations time 0.002, Idiot 0.00

Starting MIP optimization


<OptimizationStatus.OPTIMAL: 0>

### 結果を取得

In [7]:
df_books["Val_x"] = df_books["Var_x"].astype(float)
df_books

,title,n_readers,price,Var_x,Val_x
0,たのしいPython,4,3000,x_0,1.0
1,やさしいNumPy,3,4000,x_1,0.0
2,難しい機械学習,2,2000,x_2,1.0


In [8]:
repr(df_books.Val_x.max())

'LinExprTensor(1.)'

In [9]:
df_books[df_books["Val_x"] > 0.5]["title"].to_list()

['たのしいPython', '難しい機械学習']

## 一連の処理を関数化

In [10]:
def optimize_book_to_buy(name, money):
    # 既読の書籍
    name_titles_set = df.loc[df["name"] == name, "title"].unique()
    # 未読の書籍
    df_filtered = df[~df["title"].isin(name_titles_set)]

    # 書籍ごとのタイトル、価格、読んだ人数を集計
    df_books = (
        df_filtered.groupby("title")
        .agg({"name": "nunique", "price": "first"})
        .reset_index()
        .rename(columns={"name": "n_readers"})
    )

    # モデルの作成と変数定義
    m = Model()
    # 変数: 書籍をおすすめする=1, しない=0
    df_books["Var_x"] = m.add_var_tensor((len(df_books),), "x", var_type="B")
    # 目的関数: 同僚の多くが読んでいる書籍をなるべく多くおすすめする
    m.objective = maximize(xsum(df_books["n_readers"] * df_books["Var_x"]))
    # 制約条件: 金額が予算以内
    m += xsum(df_books["price"] * df_books["Var_x"]) <= money

    # 最適化の実行
    m.optimize()

    # 結果の取得
    df_books["Val_x"] = df_books["Var_x"].astype(float)
    # 誤差による不具合を防ぐため0.5を境界に比較
    return df_books[df_books["Val_x"] > 0.5]["title"].to_list()

## 全員に対して、1000円刻みで結果を取得

In [11]:
results = []
for name in df["name"].unique():
    for money in [1000, 2000, 3000, 4000, 5000]:
        book_to_buy = optimize_book_to_buy(name, money)
        results.append((name, money, book_to_buy))
result_df = pd.DataFrame(results, columns=["name", "money", "book_to_buy"])
result_df

Cgl0004I processed model has 1 rows, 3 columns (3 integer (3 of which binary)) and 3 elements
Coin3009W Conflict graph built in 0.000 seconds, density: 23.810%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0045I Nauty did not find any useful orbits in time 2.1e-05
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -6
Cbc0038I Before mini branch and bound, 3 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I After 0.00 seconds - Feasibility pump exiting with objective of -6 - took 0.00 seconds
Cbc0012I Integer solution of -6 found by feasibility pump after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective -6, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Starting solution of the Linear programming rela

,name,money,book_to_buy
0,yukie,1000,[おいしいPython]
1,yukie,2000,[おいしいPython]
2,yukie,3000,[おいしいPython]
3,yukie,4000,[おいしいPython]
4,yukie,5000,[おいしいPython]
5,altnight,1000,[おいしいPython]
6,altnight,2000,[難しい機械学習]
7,altnight,3000,"[おいしいPython, 難しい機械学習]"
8,altnight,4000,"[おいしいPython, かんたんDjango]"
9,altnight,5000,"[かんたんDjango, 難しい機械学習]"
